In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
subset_test_path = './new_train/train_subset'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [7]:
batch_sz = 4

def train_agents_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    out = numpy.concatenate([numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def train_all_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:]]) for scene in batch])
    out = numpy.concatenate([numpy.dstack([scene['p_out'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]
    
def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
train_agent_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_agents_collate, num_workers=0)

train_all_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_all_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = True, collate_fn=test_collate, num_workers=0)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    def __init__(self,
                 input_size = 2,
                 embedding_size = 8,
                 hidden_size = 16):
        
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear(x))
        hidden = self.lstm(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    def __init__(self, embedding_size=8, hidden_size=16, output_size=2):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear1(x))
        hidden = self.lstm(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden


In [5]:
from tqdm import tqdm_notebook as tqdm

def train(encoder, decoder, device, train_loader, encoder_optimizer, decoder_optimizer, epoch, log_interval=10000):    
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(train_loader):
        
        inp, out = sample_batch
#         print(inp.shape)
        # preprocessing more ????
#         inp = inp[:,0,:,:]
#         out = out[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(inp.shape[0]):
            x_offset.append(inp[i][0][0].detach().clone())
            y_offset.append(inp[i][0][1].detach().clone())
    
        for j in range(inp.shape[0]):
            for i in range(inp.shape[1]):
                inp[j][i][0] = inp[j][i][0] - x_offset[j]
                inp[j][i][1] = inp[j][i][1] - y_offset[j]

        #output whatever
        for j in range(out.shape[0]):
            for i in range(out.shape[1]):
                out[j][i][0] = out[j][i][0] - x_offset[j]
                out[j][i][1] = out[j][i][1] - y_offset[j]
        
        _input, target = inp.to(device), out.to(device)
        
        encoder.train()
        decoder.train()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
        
        loss = 0
        
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += torch.sqrt(criterion(decoder_output[:, :2], target[:, di, :2]))

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / 30

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
#         file1 = open("loss_steps.txt", "a")  # append mode
#         file1.write(str(loss.item()) + ",")
#         file1.close()
        
#       output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        iterator.set_postfix(loss=(loss.item()))
    

In [6]:
device = "cuda"
encoder = EncoderRNN(input_size=2)
decoder = DecoderRNN(output_size=2)

encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)

encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

num_epoch = 6

for epoch in range(1, num_epoch + 1):
    train(encoder, decoder, device, train_all_loader, encoder_optimizer, decoder_optimizer, epoch)
#         train(encoder, decoder, device, train_agent_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)

In [7]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.
    
    referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting
    
    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
    
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
#         _input = _input[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(_input.shape[0]):
            x_offset.append(_input[i][0][0].detach().clone())
            y_offset.append(_input[i][0][1].detach().clone())
    
        for j in range(_input.shape[0]):
            for i in range(_input.shape[1]):
                _input[j][i][0] = _input[j][i][0] - x_offset[j]
                _input[j][i][1] = _input[j][i][1] - y_offset[j]

        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 2)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        for i in range(30):
            for j in range(batch_size):
                decoder_outputs[j,i,0] = decoder_outputs[j,i,0] + x_offset[j]
                decoder_outputs[j,i,1] = decoder_outputs[j,i,1] + y_offset[j]
            
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:].tolist())
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:].tolist()]
                
    return(forecasted_trajectories)

In [8]:
output = infer_absolute(test_loader, encoder, decoder)

In [9]:
import pandas as pd
df = pd.DataFrame.from_dict(output, orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
29158,"[1761.9212646484375, 387.7587890625]","[1761.08447265625, 387.07843017578125]","[1760.232421875, 386.38702392578125]","[1759.4483642578125, 385.6621398925781]","[1758.636474609375, 384.93438720703125]","[1757.81396484375, 384.20489501953125]","[1756.98046875, 383.473876953125]","[1756.1357421875, 382.7418518066406]","[1755.27978515625, 382.0090637207031]","[1754.4132080078125, 381.2756652832031]",...,"[1744.737548828125, 373.3222961425781]","[1743.9029541015625, 372.6307678222656]","[1743.0802001953125, 371.94659423828125]","[1742.2679443359375, 371.269287109375]","[1741.464599609375, 370.5981750488281]","[1740.66845703125, 369.9324035644531]","[1739.8775634765625, 369.2711181640625]","[1739.09033203125, 368.6135559082031]","[1738.304931640625, 367.9591369628906]","[1737.5203857421875, 367.3074035644531]"
24937,"[1877.9931640625, 480.5816650390625]","[1877.1485595703125, 479.7559509277344]","[1876.228271484375, 478.98974609375]","[1875.381591796875, 478.19964599609375]","[1874.5062255859375, 477.408203125]","[1873.61669921875, 476.6156921386719]","[1872.7137451171875, 475.8226013183594]","[1871.798095703125, 475.0293884277344]","[1870.871337890625, 474.236328125]","[1869.93505859375, 473.4437255859375]",...,"[1859.6904296875, 464.8797912597656]","[1858.8116455078125, 464.1278076171875]","[1857.94287109375, 463.3810119628906]","[1857.082763671875, 462.63916015625]","[1856.230224609375, 461.902099609375]","[1855.3839111328125, 461.1697082519531]","[1854.5430908203125, 460.4421081542969]","[1853.706787109375, 459.7196044921875]","[1852.8746337890625, 459.0025634765625]","[1852.0460205078125, 458.2915954589844]"
28080,"[599.8182373046875, 1520.7572021484375]","[600.2845458984375, 1520.9400634765625]","[600.8784790039062, 1521.3505859375]","[601.3313598632812, 1521.6083984375]","[601.7942504882812, 1521.92822265625]","[602.2666625976562, 1522.2628173828125]","[602.7610473632812, 1522.6129150390625]","[603.2811889648438, 1522.969970703125]","[603.8255615234375, 1523.329345703125]","[604.389404296875, 1523.6884765625]",...,"[610.4613037109375, 1527.8641357421875]","[610.988525390625, 1528.28515625]","[611.5216674804688, 1528.7115478515625]","[612.0626831054688, 1529.1422119140625]","[612.6129150390625, 1529.5760498046875]","[613.1729125976562, 1530.011962890625]","[613.7426147460938, 1530.44873046875]","[614.3212280273438, 1530.885498046875]","[614.907470703125, 1531.3212890625]","[615.4994506835938, 1531.75537109375]"
18089,"[1880.7557373046875, 486.6918029785156]","[1879.950439453125, 486.1208801269531]","[1879.1513671875, 485.496826171875]","[1878.418212890625, 484.8328552246094]","[1877.6611328125, 484.166259765625]","[1876.8988037109375, 483.4985046386719]","[1876.1302490234375, 482.8296203613281]","[1875.3543701171875, 482.1600646972656]","[1874.5706787109375, 481.4900817871094]","[1873.7794189453125, 480.81988525390625]",...,"[1865.007568359375, 473.6301574707031]","[1864.257568359375, 473.018310546875]","[1863.5191650390625, 472.4151611328125]","[1862.791015625, 471.81982421875]","[1862.0711669921875, 471.23101806640625]","[1861.3572998046875, 470.6473388671875]","[1860.6473388671875, 470.06732177734375]","[1859.93896484375, 469.4895935058594]","[1859.23046875, 468.9129333496094]","[1858.52001953125, 468.3363037109375]"
3877,"[1968.7879638671875, 556.7788696289062]","[1967.55517578125, 555.7265014648438]","[1966.4052734375, 554.7474975585938]","[1965.29833984375, 553.764404296875]","[1964.1629638671875, 552.784912109375]","[1963.0118408203125, 551.8080444335938]","[1961.84912109375, 550.8348388671875]","[1960.6796875, 549.8656616210938]","[1959.507568359375, 548.9005126953125]","[1958.3359375, 547.9387817382812]",...,"[1945.64208984375, 537.2162475585938]","[1944.51416015625, 536.2277221679688]","[1943.4002685546875, 535.2503662109375]","[1942.3035888671875, 534.2898559570312]","[1941.2266845703125, 533.3514404296875]","[1940.1710205078125, 532.439697265625]","[193

In [10]:
for i in range(30):
    df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)

In [11]:
import numpy as np
dropped_cols = list(np.arange(30))
df2 = df.drop(dropped_cols, axis=1)
df2.index.name = 'ID'
df2

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
ID,,,,,,,,,,,,,,,,,,,,,
29158,1761.921265,387.758789,1761.084473,387.078430,1760.232422,386.387024,1759.448364,385.662140,1758.636475,384.934387,...,1740.668457,369.932404,1739.877563,369.271118,1739.090332,368.613556,1738.304932,367.959137,1737.520386,367.307404
24937,1877.993164,480.581665,1877.148560,479.755951,1876.228271,478.989746,1875.381592,478.199646,1874.506226,477.408203,...,1855.383911,461.169708,1854.543091,460.442108,1853.706787,459.719604,1852.874634,459.002563,1852.046021,458.291595
28080,599.818237,1520.757202,600.284546,1520.940063,600.878479,1521.350586,601.331360,1521.608398,601.794250,1521.928223,...,613.172913,1530.011963,613.742615,1530.448730,614.321228,1530.885498,614.907471,1531.321289,615.499451,1531.755371
18089,1880.755737,486.691803,1879.950439,486.120880,1879.151367,485.496826,1878.418213,484.832855,1877.661133,484.166260,...,1861.357300,470.647339,1860.647339,470.067322,1859.938965,469.489594,1859.230469,468.912933,1858.520020,468.336304
3877,1968.787964,556.778870,1967.555176,555.726501,1966.405273,554.747498,1965.298340,553.764404,1964.162964,552.784912,...,1940.171021,532.439697,1939.136719,531.557861,1938.122925,530.707642,1937.127808,529.888977,1936.148926,529.099854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33776,747.686340,1388.255615,747.548828,1388.911255,747.344971,1390.027588,747.206604,1390.999878,747.109863,1391.963135,...,745.962219,1413.606201,745.878113,1414.522217,745.784851,1415.414429,745.679382,1416.282715,745.557495,1417.127686
3825,258.843079,862.507080,258.951782,861.484924,258.998535,860.781189,259.013702,860.010986,259.037170,859.253601,...,259.707428,844.273193,259.737579,843.602295,259.764679,842.933105,259.788361,842.264771,259.808350,841.596436
31990,1792.318848,409.265625,1791.307495,408.410400,1790.326172,407.567627,1789.388306,406.712830,1788.421387,405.858002,...,1767.530029,388.290649,1766.607300,387.492371,1765.693359,386.704773,1764.788452,385.929688,1763.892700,385.168671


In [12]:
df2.to_csv("outputs6epb4all.csv", index=True)